In [1]:
# import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import collections
from datetime import datetime
# from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
373,tt0947798,13000000,327803731,Black Swan,Natalie Portman|Mila Kunis|Vincent Cassel|Barb...,Darren Aronofsky,"In the era of personal branding, the scariest ...","A ballet dancer wins the lead in ""Swan Lake"" a...",108,Drama|Mystery|Thriller,Fox Searchlight Pictures|Dune Entertainment|Pr...,12/2/2010,7.1,2010
992,tt1386703,125000000,198802074,Total Recall,Colin Farrell|Kate Beckinsale|Jessica Biel|Bry...,Len Wiseman,What Is Real?,"Welcome to Rekall, the company that can turn y...",121,Action|Science Fiction|Adventure|Thriller,Columbia Pictures,8/2/2012,5.8,2012
269,tt1114740,26000000,183293131,Paul Blart: Mall Cop,Kevin James|Keir O'Donnell|Jayma Mays|Raini Ro...,Steve Carr,Safety Never Takes A Holiday.,Mild-mannered Paul Blart (Kevin James) has alw...,91,Action|Adventure|Comedy|Family,Columbia Pictures|Happy Madison Productions|Re...,1/15/2009,5.4,2009
1383,tt0342258,45000000,50871113,Unleashed,Jet Li|Morgan Freeman|Bob Hoskins|Vincent Rega...,Louis Leterrier,Serve No Master,"Raised as a slave, Danny is used to fighting f...",103,Action|Crime,Canal Plus|Current Entertainment|Clubdeal|TF1 ...,2/2/2005,6.5,2005
22,tt2126355,110000000,470490832,San Andreas,Dwayne Johnson|Alexandra Daddario|Carla Gugino...,Brad Peyton,"A rescue pilot survived an earthquake, this is...",In the aftermath of a massive earthquake in Ca...,114,Action|Drama|Thriller,New Line Cinema|Village Roadshow Pictures|Warn...,5/27/2015,6.1,2015


# Предобработка

In [3]:
answers = {}    # создадим словарь для ответов
    
data['release_month'] = data.apply(lambda row: datetime.strptime(row['release_date'], '%m/%d/%Y').month, axis=1)
data['profit'] = data.apply(lambda row: row['revenue'] - row['budget'], axis=1)

genres_list = list(pd.Series(data.genres.str.split('|').sum()).value_counts().index)    # список жанров
actors_list = list(pd.Series(data.cast.str.split('|').sum()).value_counts().index)    # список актеров
directors_list = list(pd.Series(data.director.str.split('|').sum()).value_counts().index)    # список режисерров
prod_comp_list = list(pd.Series(data.production_companies.str.split('|').sum()).value_counts().index)    # список студий

'''DataFrame с актерами, которые играли вместе в одном фильме'''
cast_match_df = pd.DataFrame(0, columns=actors_list, index=actors_list)
for actor in actors_list:
    for casts in data[data.cast.str.contains(actor)].cast:
        for cast in casts.split('|'):
            if cast != actor:
                cast_match_df[actor][cast] += 1

                
def format_film_name(film):
    '''Принимает DataFrame с фильмом, возвращает string с названием фильма и IMDB ID'''
    return '{} ({})'.format(film.iloc[0]['original_title'], film.iloc[0]['imdb_id'])


def group_by_multiple_values_sum(df, group_by, value_row, split_by='|'):
    '''Раскладывает множественные значения в ячейке, группирует по этим значениям, 
    выдает отсортированный по суммам DF'''
    cleaned_df = df.set_index(value_row)[group_by].str.split(split_by, expand=True).stack().reset_index(1, drop=True)
    grouped_df = cleaned_df.reset_index(name=group_by).groupby(by=group_by).sum()
    return grouped_df.sort_values(value_row, ascending=False)


def group_by_multiple_values_mean(df, group_by, value_row, split_by='|'):
    '''Раскладывает множественные значения в ячейке, группирует по этим значениям, 
    выдает отсортированный по среднему значению DF'''
    cleaned_df = df.set_index(value_row)[group_by].str.split(split_by, expand=True).stack().reset_index(1, drop=True)
    grouped_df = cleaned_df.reset_index(name=group_by).groupby(by=group_by).mean()
    return grouped_df.sort_values(value_row, ascending=False)

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [4]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [5]:
answer_1 = data.loc[data.budget == data.budget.max(), ['original_title', 'imdb_id']]
'{} ({})'.format(answer_1.iloc[0][0], answer_1.iloc[0][1])
del answer_1

ВАРИАНТ 2

In [6]:
format_film_name(data[data.budget == data.budget.max()])

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [7]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [8]:
format_film_name(data[data.runtime == data.runtime.max()])

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?

In [9]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [10]:
format_film_name(data[data.runtime == data.runtime.min()])

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?

In [11]:
answers['4'] = 110

In [12]:
int(round(data.runtime.mean(), 0))

110

# 5. Каково медианное значение длительности фильмов?

In [13]:
answers['5'] = 107

In [14]:
int(round(data.runtime.median(), 0))

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [15]:
answers['6'] = 'Avatar (tt0499549)'

In [16]:
format_film_name(data[data.profit == data.profit.max()])

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный?

In [17]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [18]:
format_film_name(data[data.profit == data.profit.min()])

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [19]:
answers['8'] = 1478

In [20]:
data[data.profit > 0].imdb_id.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [21]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [22]:
films_2008 = data[data.release_year == 2008]
format_film_name(films_2008[films_2008.revenue == films_2008.revenue.max()])

'The Dark Knight (tt0468569)'

ВАРИАНТ 2

In [23]:
format_film_name(data[data.release_year == 2008].query('revenue == revenue.max()'))

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [24]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [25]:
films_2012_2014 = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
format_film_name(films_2012_2014[films_2012_2014.profit == films_2012_2014.profit.min()])

'The Lone Ranger (tt1210819)'

ВАРИАНТ 2

In [26]:
format_film_name(data[(data.release_year >= 2012) & (data.release_year <= 2014)].query('profit == profit.min()'))

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [27]:
answers['11'] = 'Drama'

In [28]:
pd.Series(data.genres.str.split('|').sum()).value_counts().index[0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [29]:
answers['12'] = 'Drama'

In [30]:
pd.Series(data[data.profit > 0].genres.str.split('|').sum()).value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [31]:
answers['13'] = 'Peter Jackson'

In [32]:
biggest_revenue = 0
most_valued_director = ''
for director in directors_list:
    revenue_sum = data[data.director.str.contains(director)].revenue.sum()
    if revenue_sum > biggest_revenue:
        biggest_revenue = revenue_sum
        most_valued_director = director
print(most_valued_director)
del biggest_revenue, most_valued_director, director, revenue_sum

Peter Jackson


ВАРИАНТ 2

In [33]:
cleaned = data.set_index('revenue').director.str.split('|', expand=True).stack().reset_index(1, drop=True)
grouped = cleaned.reset_index(name='director').groupby(by='director').sum()
grouped.sort_values('revenue', ascending=False).index[0]

'Peter Jackson'

ВАРИАНТ 3

In [34]:
group_by_multiple_values_sum(data, 'director', 'revenue').index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [35]:
answers['14'] = 'Robert Rodriguez'

In [36]:
pd.Series(data[data.genres.str.contains('Action')].director.str.split('|').sum()).value_counts().index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [37]:
answers['15'] = 'Chris Hemsworth'

In [38]:
group_by_multiple_values_sum(data[data.release_year == 2012], 'cast', 'revenue').index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [39]:
answers['16'] = 'Matt Damon'

In [40]:
pd.Series(data[data.budget > data.budget.mean()].cast.str.split('|').sum()).value_counts().index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [41]:
answers['17'] = 'Action'

In [42]:
pd.Series(data[data.cast.str.contains('Nicolas Cage')].genres.str.split('|').sum()).value_counts().index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [43]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [44]:
paramount = data[data.production_companies.str.contains('Paramount Pictures')]
format_film_name(paramount[paramount.profit == paramount.profit.min()])

'K-19: The Widowmaker (tt0267626)'

ВАРИАНТ 2

In [45]:
format_film_name(data.query('production_companies.str.contains("Paramount Pictures")').query('profit == profit.min()'))

'K-19: The Widowmaker (tt0267626)'

ВАРИАНТ 3

In [46]:
format_film_name(data[data.production_companies.
                      map(lambda x: True if 'Paramount Pictures' in x else False)].query('profit == profit.min()'))

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [47]:
answers['19'] = '2015'

In [48]:
data.groupby(by='release_year').sum().sort_values('revenue', ascending=False).index[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [49]:
answers['20'] = '2014'

In [50]:
data[data.production_companies.str.contains('Warner Bros')].groupby(
    by='release_year').sum().sort_values('profit', ascending=False).index[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [51]:
answers['21'] = '9'

In [52]:
data.groupby(by='release_month').imdb_id.count().sort_values(ascending=False).index[0]

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [53]:
answers['22'] = '450'

In [54]:
data[(data.release_month == 6) | (data.release_month == 7) | (data.release_month == 8)].imdb_id.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [55]:
answers['23'] = 'Peter Jackson'

In [56]:
pd.Series(data[(data.release_month == 12) | (data.release_month == 1) | (data.release_month == 2)].
          director.str.split('|').sum()).value_counts().index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [57]:
answers['24'] = 'Four By Two Productions'

In [58]:
title_len = data
title_len['title_len'] = data.apply(lambda row: len(row['original_title']), axis=1)
group_by_multiple_values_mean(title_len, 'production_companies', 'title_len').index[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [59]:
answers['25'] = 'Midnight Picture Show'

In [60]:
overview_word_len = data
overview_word_len['overview_word_len'] = data.apply(lambda row: len(row['overview'].split()), axis=1)
group_by_multiple_values_mean(overview_word_len, 'production_companies', 'overview_word_len').index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [61]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [62]:
top_vote_films = list(data.nlargest(int(data.imdb_id.count()/100), 'vote_average').original_title)

variants_26 = {0: 'Inside Out, The Dark Knight, 12 Years a Slave',
               1: 'BloodRayne, The Adventures of Rocky & Bullwinkle',
               2: 'Batman Begins, The Lord of the Rings: The Return of the King, Upside Down',
               3: '300, Lucky Number Slevin, Kill Bill: Vol. 1', 
               4: 'Upside Down, Inside Out, Iron Man'}

for key, value in variants_26.items():
    if set(value.split(', ')).issubset(set(top_vote_films)):
        print(value)

Inside Out, The Dark Knight, 12 Years a Slave


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [63]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

In [64]:
a = list(cast_match_df[cast_match_df.max() == cast_match_df.max().max()].index)
print(' & '.join(a))

Emma Watson & Daniel Radcliffe & Rupert Grint


ВАРИАНТ 2

In [65]:
variants_27 = {0: 'Johnny Depp & Helena Bonham Carter', 
               1: 'Ben Stiller & Owen Wilson', 
               2: 'Vin Diesel & Paul Walker',
               3: 'Adam Sandler & Kevin James', 
               4: 'Daniel Radcliffe & Rupert Grint'}

match = 0
variant = 0
for key, value in variants_27.items():
    match_value = cast_match_df[value.split(' & ')[0]][value.split(' & ')[1]]
    if match < match_value:
        match = match_value
        variant = key
print(variants_27[variant])


Daniel Radcliffe & Rupert Grint


# Submission

In [66]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': '9',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [67]:
# и убедиться что ни чего не пропустил)
len(answers)

27